### Please follow the instructions given in the class

XGBoost

In [1]:
#Import XGBoost, Pandas, and sklearn for the function that we will use to calculate the accuracy. 
#The accuracy is required to understand how our model is performing.

from sklearn.model_selection import train_test_split 
import pandas as pd  
from sklearn.metrics import accuracy_score
import numpy as np
import xgboost as xgb

In [2]:
#Import the wholesale customer dataset - 1 point

data = pd.read_csv("wholesale-data.csv")
X = data.iloc[:, 1:]
y = data.iloc[:, 0:1]
features=X.columns.values
target=y.columns.values  

In [3]:
#Create training and test sets - 80:20 - 1 point
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
#Convert the pandas dataframe into a DMatrix, an internal data structure that is used by XGBoost to store training and testing datasets.
# - 2 points
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test) 

In [29]:
#Specify the training parameters and train the model. - 4 points

# option 1
param = {
    'learning_rate': 0.1,
    'max_depth': 5,  # the maximum depth of each tree
    'eta': 0.01,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3}  # the number of classes that exist in this datset 
num_round = 1000
booster = xgb.train(param, dtrain, num_round)


# option 2
xgbcli = xgb.XGBClassifier(learning_rate =0.1,
                           n_estimators=1000, 
                           max_depth=6, 
                           min_child_weight=1, 
                           gamma=0, 
                           subsample=0.8, 
                           colsample_bytree=0.8, 
                           objective= 'multi:softprob',
                           num_class= 3,
                           nthread=4, 
                           scale_pos_weight=1,
                           seed=27) 
 
#xgbcli.fit(X_train, y_train) 
gb_param = xgbcli.get_xgb_params() 
cvresult = xgb.cv(gb_param, dtrain, num_boost_round=xgbcli.get_params()['n_estimators'], nfold=5, metrics='mlogloss', early_stopping_rounds=50, verbose_eval=False)
xgbcli.set_params(n_estimators=cvresult.shape[0])

xgbcli.fit(X_train, y_train, eval_metric='mlogloss')



'''
OBJECTIVE 
- binary:logistic target should be 0 or 1, but this case target 1 or 2.
- multi:softmax: set XGBoost to do multiclass classification using the softmax objective, you also need to set num_class(number of classes)
- multi:softprob: same as softmax, but output a vector of ndata * nclass, which can be further reshaped to ndata * nclass matrix. The result contains predicted probability of each data point belonging to each class.

metrics
- mlogloss :  Log loss penalizes false classifications by taking into account the probability of classification. 
- merror: Multiclass classification error rate. It is calculated as #(wrong cases)/#(all cases).
'''



/Users/myungjong/opt/anaconda3/envs/p37env/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


'\nOBJECTIVE \n- binary:logistic target should be 0 or 1, but this case target 1 or 2.\n- multi:softmax: set XGBoost to do multiclass classification using the softmax objective, you also need to set num_class(number of classes)\n- multi:softprob: same as softmax, but output a vector of ndata * nclass, which can be further reshaped to ndata * nclass matrix. The result contains predicted probability of each data point belonging to each class.\n\nmetrics\n- mlogloss :  Log loss penalizes false classifications by taking into account the probability of classification. \n- merror: Multiclass classification error rate. It is calculated as #(wrong cases)/#(all cases).\n'

In [30]:
#Predict the "Channel" values of the test set using the model that we just created. - 1 point
preds = booster.predict(dtest)
y_pred = xgbcli.predict(X_test) 

In [31]:
#Get the accuracy of the model that we have trained for the test dataset. - 1 point
from sklearn.metrics import precision_score, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

print("1 option 1's result: \n")
best_preds = np.asarray([np.argmax(line) for line in preds])
#print(best_preds)
pre_score = precision_score(y_test, best_preds, average='macro')
print("pre_score: {0}".format(pre_score*100))
acc_score = accuracy_score(y_test , best_preds)
print("acc_score: {0}".format(acc_score*100))
 

print("\n\n\n 2. option 2's result: ")
print(classification_report(y_test, y_pred, target_names=['c1','c2']))

scores = cross_val_score(xgbcli, X_train, y_train.values.ravel(), cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

1 option 1's result: 

pre_score: 89.42857142857143
acc_score: 91.81818181818183



 2. option 2's result: 
              precision    recall  f1-score   support

          c1       0.97      0.90      0.93        78
          c2       0.79      0.94      0.86        32

    accuracy                           0.91       110
   macro avg       0.88      0.92      0.90       110
weighted avg       0.92      0.91      0.91       110

Mean cross-validation score: 0.92
